<a href="https://colab.research.google.com/github/remphan1618/liteOutInpainta/blob/main/IOPIAINT%20FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#@title 🎨 Model Lister



from huggingface_hub import list_models
from huggingface_hub.utils import HfHubHTTPError # Import for better error handling

# Define the filters
try:
    models_generator = list_models(
        pipeline_tag="text-to-image",
        library="diffusers",
        search="inpaint",
        sort="lastModified", # Sort by last update time
        direction=-1,       # -1 for descending (newest first)
    )

    # Extract the repo_ids
    inpaint_model_ids = [model.modelId for model in models_generator]

    # --- Print model IDs 10 per line, space-separated ---
    print("List of Inpaint Model IDs (10 per line, space-separated):")
    if inpaint_model_ids:
        # Print model IDs in blocks of 10 per line
        for i in range(0, len(inpaint_model_ids), 10):
            # Get a slice of 10 model IDs
            batch = inpaint_model_ids[i:i + 10]
            # Join them with a space and print
            print(" ".join(batch))
    else:
        print("No models found matching the criteria.")

    # --- Print model IDs in chunks, comma-separated with trailing comma ---
    comma_chunk_size = 5 # You can change this to 10 if you prefer
    print(f"\nComma-separated list ({comma_chunk_size} per line, with space and trailing comma):")
    if inpaint_model_ids:
        # Print model IDs in blocks of 'comma_chunk_size' per line
        for i in range(00, len(inpaint_model_ids), comma_chunk_size): # Start from 0
            # Get a slice of 'comma_chunk_size' model IDs
            batch = inpaint_model_ids[i:i + comma_chunk_size]
            # Join them with a comma and a space
            line = ", ".join(batch)

            # Check if this is the last batch
            is_last_batch = (i + comma_chunk_size) >= len(inpaint_model_ids)

            if not is_last_batch:
                # If not the last batch, print the line followed by a comma and newline
                print(line + ",") # Add trailing comma
            else:
                # If it is the last batch, just print the line
                print(line)
    else:
         print(f"\nNo models found to create a comma-separated list ({comma_chunk_size} per line).")


except HfHubHTTPError as e:
    print(f"An HTTP error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

An HTTP error occurred: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models?filter=diffusers&pipeline_tag=text-to-image&search=inpaint&sort=lastModified&direction=-1 (Request ID: Root=1-6811643c-7a49788d6d3e8bb04f72275a;5dcd2abd-293e-489c-9284-965ff5b0936b)

Invalid credentials in Authorization header


In [6]:
#@title 🎨 All-in-One IOpaint with Ngrok

#@markdown ## 🔑 Ngrok Configuration
#@markdown Enter your [ngrok auth token](https://dashboard.ngrok.com/get-started/your-authtoken) below:
ngrok_token = "2tjxIXifSaGR3dMhkvhk6sZqbGo_6ZfBZLZHMbtAjfRmfoDW5" #@param {type:"string"}
save_token = True #@param {type:"boolean"}
token_file = "/content/ngrok_token" #@param {type:"string"}
run_without_ngrok = False #@param {type:"boolean"}

#@markdown ## 📦 Model Selection
#@markdown ### Checkpoint Models (comma-separated list)
checkpoint_models = "lama, sam2_base, gfpgan" #@param {type:"string"}

#@markdown ### Diffusers Models (comma-separated list of HF repos)
diffusers_models = "" #@param {type:"string"}
use_fp16_for_diffusers = True #@param {type:"boolean"}
pre_download_diffusers = False #@param {type:"boolean"}

#@markdown ## 🚀 IOpaint Options
selected_model = "lama" #@param ["lama", "mat", "migan", "sdxl_turbo", "urpm_inpaint_sdxl"]
enable_interactive_seg = True #@param {type:"boolean"}
enable_remove_bg = True #@param {type:"boolean"}
enable_realesrgan = True #@param {type:"boolean"}
enable_gfpgan = True #@param {type:"boolean"}
disable_nsfw = True #@param {type:"boolean"}
port = 1111 #@param {type:"integer"}

import os
import sys
import json
import time
import subprocess
import requests
from datetime import datetime
from tqdm.notebook import tqdm
from IPython.display import clear_output, display, HTML

# Current timestamp
print(f"Current Date and Time: {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC")
print(f"Current User: remphan1618")

# Define base directories - Using /content/ for user accessibility
BASE_MODEL_DIR = "/content/iopaint/models"
TORCH_HUB_DIR = os.path.join(BASE_MODEL_DIR, "torch/hub/checkpoints")
os.makedirs(BASE_MODEL_DIR, exist_ok=True)
os.makedirs(TORCH_HUB_DIR, exist_ok=True)

# ========== INSTALL REQUIRED PACKAGES ==========
print("\n📦 Installing required packages...")

# Install packages one by one to better handle failures
def install_package(package):
    try:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"✅ Successfully installed {package}")
        return True
    except Exception as e:
        print(f"❌ Failed to install {package}: {str(e)}")
        return False

# Critical packages first
install_package("iopaint")

# Install pyngrok only if we're going to use it
if not run_without_ngrok:
    install_package("pyngrok")

# Other required/optional packages
install_package("huggingface_hub")
if enable_gfpgan:
    install_package("gfpgan")
if enable_remove_bg:
    install_package("rembg")
    install_package("onnxruntime")

# Create symlink from default location to our accessible location
default_model_dir = "/root/.cache/iopaint/models"
os.makedirs(os.path.dirname(default_model_dir), exist_ok=True)
if os.path.exists(default_model_dir):
    if os.path.islink(default_model_dir):
        os.unlink(default_model_dir)
    else:
        import shutil
        print(f"Moving existing models from {default_model_dir} to {BASE_MODEL_DIR}...")
        for item in os.listdir(default_model_dir):
            src = os.path.join(default_model_dir, item)
            dst = os.path.join(BASE_MODEL_DIR, item)
            if not os.path.exists(dst):
                if os.path.isdir(src):
                    shutil.copytree(src, dst)
                else:
                    shutil.copy2(src, dst)
        shutil.rmtree(default_model_dir)

# Create symlink
os.symlink(BASE_MODEL_DIR, default_model_dir)
print(f"✅ Models directory set to {BASE_MODEL_DIR} (accessible via /content/)")
print(f"✅ Created symlink from {default_model_dir} to {BASE_MODEL_DIR}")

# ========== HANDLE NGROK TOKEN ==========
use_ngrok = False
if not run_without_ngrok:
    # Load token from file if exists
    if not ngrok_token and os.path.exists(token_file):
        with open(token_file, 'r') as f:
            ngrok_token = f.read().strip()
        print(f"✅ Loaded Ngrok token from {token_file}")

    # Save token if requested
    if ngrok_token and save_token:
        with open(token_file, 'w') as f:
            f.write(ngrok_token)
        os.chmod(token_file, 0o600)  # Set permissions to user-only
        print(f"✅ Saved Ngrok token to {token_file}")

    if not ngrok_token:
        print("⚠️ WARNING: No Ngrok token provided.")
        print("Will continue without Ngrok - you'll only be able to access IOpaint through Colab's port forwarding.")
    else:
        use_ngrok = True
else:
    print("📌 Running without Ngrok as requested.")

# ========== CHECKPOINT MODEL DOWNLOADER ==========
# Checkpoint models database
CHECKPOINT_MODELS = {
    "lama": {
        "url": "https://github.com/Sanster/models/releases/download/add_big_lama/big-lama.pt",
        "filename": "big-lama.pt",
        "size_mb": 196,
        "description": "Default fast inpainting model"
    },
    "mat": {
        "url": "https://github.com/Sanster/models/releases/download/mat/Places_512_FullData_G.pth",
        "filename": "Places_512_FullData_G.pth",
        "size_mb": 675,
        "description": "Better quality than LaMa but slower"
    },
    "sam2_base": {
        "url": "https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt",
        "filename": "sam2_hiera_base_plus.pt",
        "size_mb": 309,
        "description": "SAM2 model for interactive segmentation"
    },
    "gfpgan": {
        "url": "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth",
        "filename": "GFPGANv1.3.pth",
        "size_mb": 332,
        "description": "Face restoration model"
    },
    "migan": {
        "url": "https://github.com/Sanster/models/releases/download/migan/migan_patch.pt",
        "filename": "migan_patch.pt",
        "size_mb": 395,
        "description": "Another good inpainting model"
    }
}

def download_checkpoint(url, dest_path):
    """Download a file with progress bar"""
    if os.path.exists(dest_path):
        file_size_mb = os.path.getsize(dest_path) / (1024 * 1024)
        print(f"✅ Already exists: {os.path.basename(dest_path)} ({file_size_mb:.1f} MB)")
        return True

    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            total_size = int(r.headers.get('content-length', 0))

            with open(dest_path, 'wb') as f, tqdm(
                desc=os.path.basename(dest_path),
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for chunk in r.iter_content(chunk_size=8192):
                    size = f.write(chunk)
                    bar.update(size)

        if os.path.exists(dest_path) and os.path.getsize(dest_path) > 0:
            size_mb = os.path.getsize(dest_path) / (1024 * 1024)
            print(f"✅ Downloaded: {os.path.basename(dest_path)} ({size_mb:.1f} MB)")
            return True
        else:
            print(f"❌ Failed to download: {dest_path}")
            return False
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

# Process checkpoint models
if checkpoint_models:
    model_list = [m.strip() for m in checkpoint_models.split(",") if m.strip()]
    print(f"\n📦 Downloading {len(model_list)} checkpoint models: {', '.join(model_list)}")

    for model_name in model_list:
        if model_name in CHECKPOINT_MODELS:
            print(f"\n📥 Downloading {model_name}: {CHECKPOINT_MODELS[model_name]['description']}")
            dest_path = os.path.join(TORCH_HUB_DIR, CHECKPOINT_MODELS[model_name]['filename'])
            download_checkpoint(CHECKPOINT_MODELS[model_name]['url'], dest_path)
        else:
            print(f"❌ Unknown model: {model_name}")

# ========== DIFFUSERS MODEL DOWNLOADER ==========
def sanitize_name(repo_id):
    """Create a valid model name from repository ID"""
    name = repo_id.split("/")[-1].lower()
    name = name.replace("-", "_").replace(".", "_").replace(" ", "_")
    if len(name) > 30:
        name = name[:30]
    return name

def setup_diffusers_model(repo_id, use_fp16=False, pre_download=False):
    """Set up a diffusers model by creating model_info.json"""
    model_name = sanitize_name(repo_id)
    model_dir = os.path.join(BASE_MODEL_DIR, model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Create model_info.json
    config = {
        "name": model_name,
        "repo_id": repo_id,
        "subfolder": None,
        "type": "diffusers",
        "fp16": use_fp16,
        "revision": "main",
        "use_checkpoint": False  # Force diffusers format only
    }

    config_path = os.path.join(model_dir, "model_info.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=2)

    print(f"✅ Set up model: {model_name}")
    print(f"  • Repository: {repo_id}")
    print(f"  • Using FP16: {'Yes' if use_fp16 else 'No'}")

    # Optionally pre-download the model
    if pre_download:
        try:
            print(f"  • Pre-downloading diffusers components...")

            # Import huggingface_hub
            from huggingface_hub import snapshot_download

            # Only download diffusers components
            allow_patterns = [
                "*.json",
                "scheduler/**",
                "text_encoder/**",
                "text_encoder_2/**",
                "tokenizer/**",
                "unet/**",
                "vae/**",
                "safety_checker/**",
                "feature_extractor/**"
            ]

            # Skip checkpoint files
            ignore_patterns = ["*.safetensors", "*.ckpt", "*.bin", "*.pt"]

            # Download with progress tracking
            print(f"    ⏳ Downloading from {repo_id}...")
            snapshot_download(
                repo_id=repo_id,
                local_dir=model_dir,
                allow_patterns=allow_patterns,
                ignore_patterns=ignore_patterns
            )
            print(f"    ✅ Download complete!")
        except Exception as e:
            print(f"    ❌ Error pre-downloading: {str(e)}")
            print(f"    ℹ️ Model will download when first used in IOpaint.")
    else:
        print(f"  • Model will download when first used in IOpaint")

    return model_name

# Process diffusers models
configured_diffusers = []
if diffusers_models:
    model_repos = [m.strip() for m in diffusers_models.split(",") if m.strip()]
    print(f"\n🧩 Setting up {len(model_repos)} diffusers models")

    for repo_id in model_repos:
        print(f"\n📥 Processing: {repo_id}")
        model_name = setup_diffusers_model(repo_id, use_fp16_for_diffusers, pre_download_diffusers)
        configured_diffusers.append(model_name)
        time.sleep(0.5)  # Small delay between setups

# ========== NGROK & IOPAINT LAUNCHER ==========
# Set up Ngrok if we're using it
url = None
if use_ngrok:
    try:
        print("\n🔄 Setting up Ngrok...")
        from pyngrok import ngrok, conf, exception

        # Explicitly clear any previous authtoken
        conf.get_default().auth_token = None

        # Set auth token
        print(f"Setting Ngrok auth token...")
        ngrok.set_auth_token(ngrok_token)

        # Verify the token by getting API info
        try:
            print("Verifying Ngrok auth token...")
            # Try to use the API to verify the token
            tunnels = ngrok.get_tunnels()
            print("✅ Ngrok token is valid")
        except exception.PyngrokNgrokError as e:
            print(f"❌ Ngrok authentication failed: {str(e)}")
            print("🔄 Falling back to local-only access")
            use_ngrok = False

        if use_ngrok:
            # Create tunnel
            print(f"Creating Ngrok tunnel on port {port}...")
            url = ngrok.connect(addr=port, bind_tls=True)
            print(f"✅ Ngrok tunnel created: {url}")
    except ImportError:
        print("❌ Failed to import pyngrok. Falling back to local-only access.")
        use_ngrok = False
    except Exception as e:
        print(f"❌ Error setting up Ngrok: {str(e)}")
        print("🔄 Falling back to local-only access")
        use_ngrok = False

# Display URL info based on what we're using
if use_ngrok and url:
    # Display clickable Ngrok link
    display(HTML(f'''
    <div style="background-color: #f0f5ff; padding: 15px; border-radius: 10px; margin: 10px 0;">
      <h3>🎨 IOpaint is Ready!</h3>
      <p><b>Access URL:</b> <a href="{url}" target="_blank">{url}</a></p>
      <p><b>Models folder:</b> <code>/content/iopaint/models</code> (accessible in file explorer)</p>
      <p><b>Selected model:</b> <code>{selected_model}</code></p>
    </div>
    '''))
else:
    # Display info about local access
    display(HTML(f'''
    <div style="background-color: #f0f5ff; padding: 15px; border-radius: 10px; margin: 10px 0;">
      <h3>🎨 IOpaint is Running Locally</h3>
      <p><b>Access method:</b> Use the "Connect" button at the top right of the Colab window, then select "Connect to local runtime"</p>
      <p><b>Models folder:</b> <code>/content/iopaint/models</code> (accessible in file explorer)</p>
      <p><b>Selected model:</b> <code>{selected_model}</code></p>
      <p><b>Note:</b> Your IOpaint instance will be available at <a href="http://localhost:{port}" target="_blank">http://localhost:{port}</a> after connecting to the local runtime.</p>
    </div>
    '''))

# Build IOPaint command
cmd = [
    'iopaint', 'start',
    '--model', selected_model,
    '--device', 'cuda',
    '--port', str(port),
    '--host', '0.0.0.0'
]

# Add optional parameters
if enable_interactive_seg:
    cmd.extend(['--enable-interactive-seg', '--interactive-seg-device', 'cuda'])

if enable_remove_bg:
    cmd.append('--enable-remove-bg')

if enable_realesrgan:
    cmd.extend(['--enable-realesrgan', '--realesrgan-device', 'cuda'])

if enable_gfpgan:
    cmd.extend(['--enable-gfpgan', '--gfpgan-device', 'cuda'])

if disable_nsfw:
    cmd.append('--disable-nsfw-checker')

print("\n🚀 Launching IOPaint with command:")
print(' '.join(cmd))
print("\n⏳ Please wait until you see 'Application startup complete'...\n")

# Run the process and keep it alive
import asyncio

async def run_process():
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    async def read_stream(stream):
        while True:
            line = await stream.readline()
            if not line:
                break
            print(line.decode('utf-8').strip())

    await asyncio.gather(
        read_stream(process.stdout),
        read_stream(process.stderr)
    )

# Run the process
try:
    await run_process()
except KeyboardInterrupt:
    print("Stopping on user request...")
finally:
    # Clean up Ngrok tunnels if we were using them
    if use_ngrok:
        try:
            print("Cleaning up Ngrok tunnels...")
            ngrok.kill()
        except:
            pass

Current Date and Time: 2025-04-29 23:38:29 UTC
Current User: remphan1618

📦 Installing required packages...
Installing iopaint...
✅ Successfully installed iopaint
Installing pyngrok...
✅ Successfully installed pyngrok
Installing huggingface_hub...
✅ Successfully installed huggingface_hub
Installing gfpgan...
✅ Successfully installed gfpgan
Installing rembg...
✅ Successfully installed rembg
Installing onnxruntime...
✅ Successfully installed onnxruntime
✅ Models directory set to /content/iopaint/models (accessible via /content/)
✅ Created symlink from /root/.cache/iopaint/models to /content/iopaint/models
✅ Saved Ngrok token to /content/ngrok_token

📦 Downloading 3 checkpoint models: lama, sam2_base, gfpgan

📥 Downloading lama: Default fast inpainting model
✅ Already exists: big-lama.pt (196.1 MB)

📥 Downloading sam2_base: SAM2 model for interactive segmentation
✅ Already exists: sam2_hiera_base_plus.pt (308.5 MB)

📥 Downloading gfpgan: Face restoration model
✅ Already exists: GFPGANv1.3


🚀 Launching IOPaint with command:
iopaint start --model lama --device cuda --port 1111 --host 0.0.0.0 --enable-interactive-seg --interactive-seg-device cuda --enable-remove-bg --enable-realesrgan --realesrgan-device cuda --enable-gfpgan --gfpgan-device cuda --disable-nsfw-checker

⏳ Please wait until you see 'Application startup complete'...

2025-04-29 23:38:46.407 | INFO     | iopaint.runtime:setup_model_dir:81 - Model directory: /root/.cache
- Platform: Linux-6.1.123+-x86_64-with-glibc2.35
- Python version: 3.11.12
- torch: 2.6.0+cu124
- torchvision: 0.21.0+cu124
- Pillow: 9.5.0
- diffusers: 0.27.2
- transformers: 4.48.3
- opencv-python: 4.11.0.86
- accelerate: 1.6.0
- iopaint: 1.6.0
- rembg: 2.0.65
- onnxruntime: 1.21.1

E0000 00:00:1745969931.613958   14371 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745969931.620533   14371 cuda_blas.cc:1418] Unable to register cuBLAS factor

CancelledError: 